# <font color='blue'>Data Science Academy - Projeto Especial 1</font>
# <font color='blue'>Segmentação de Imagens Médicas com Deep Learning</font>
# <font color='blue'>Contornos das Imagens</font>

Referências:

Detection and segmentation of the Left Ventricle in Cardiac MRI using Deep Learning
https://arxiv.org/abs/1801.02171

A Combined Deep-Learning and Deformable-Model Approach to Fully Automatic Segmentation of the Left Ventricle in Cardiac MRI
https://arxiv.org/abs/1512.07951


In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.path import Path
import dicom
import os
import glob
from matplotlib.path import Path
import matplotlib.patches as patches
%matplotlib inline

### Abrindo Uma Imagem de Treinamento

In [ ]:
folder_path = './Data/challenge_training/SC-HF-I-1/'

In [ ]:
lstFilesDCM = []  # create an empty list
for dirName, subdirList, fileList in os.walk(folder_path):
    for filename in fileList:
        if ".dcm" in filename.lower():  # check whether the file's DICOM
            lstFilesDCM.append(os.path.join(dirName,filename))

In [ ]:
# Arquivo de Referência
RefDs = dicom.read_file(lstFilesDCM[0])

# Carrega as dimensões base
ConstPixelDims = (int(RefDs.Rows), int(RefDs.Columns), len(lstFilesDCM))
ConstPixelSpacing = (float(RefDs.PixelSpacing[0]), float(RefDs.PixelSpacing[1]), float(RefDs.SliceThickness))

In [ ]:
x = np.arange(0.0, (ConstPixelDims[0]+1)*ConstPixelSpacing[0], ConstPixelSpacing[0])
y = np.arange(0.0, (ConstPixelDims[1]+1)*ConstPixelSpacing[1], ConstPixelSpacing[1])
z = np.arange(0.0, (ConstPixelDims[2]+1)*ConstPixelSpacing[2], ConstPixelSpacing[2])

In [ ]:
# Array baseado em 'ConstPixelDims'
ArrayDicom = np.zeros(ConstPixelDims, dtype=RefDs.pixel_array.dtype)

# Percorrer todos os arquivos DICOM
# todos os arquivos precisam ter a mesma forma
for filenameDCM in lstFilesDCM[:9]:
    ds = dicom.read_file(filenameDCM)
    # Armazenar os dados da imagem bruta
    ArrayDicom[:, :, lstFilesDCM.index(filenameDCM)] = ds.pixel_array  

In [ ]:
plt.figure(dpi=300)
plt.axes().set_aspect('equal', 'datalim')
plt.set_cmap(plt.gray())
plt.pcolormesh(x, y, np.flipud(ArrayDicom[:, :, 8]))

In [ ]:
image_part = dicom.read_file('./Data/challenge_training/\
SC-HF-I-1/IM-0002-0048.dcm').pixel_array  
plt.imshow(image_part)

### Abrindo um Contorno de Treinamento

In [ ]:
folder_text_path = './Data/Sunnybrook Cardiac MR Database ContoursPart3/\
TrainingDataContours/SC-HF-I-01/contours-manual/IRCCI-expert/'
text_arr = glob.glob(folder_text_path + '*')

In [ ]:
text_path = './Data/Sunnybrook Cardiac MR Database ContoursPart3/\
TrainingDataContours/SC-HF-I-01/contours-manual/IRCCI-expert/\
IM-0001-0048-icontour-manual.txt'

In [ ]:
n = 28
folder = 'SC-N-11'
pic_arr = [k for k in glob.glob("./Data/challenge_online/challenge_online/%s/*" % folder) if '00' + str(n) in k]
text_path = './Data/Sunnybrook Cardiac MR Database ContoursPart1/\
OnlineDataContours/%s/contours-manual/IRCCI-expert/\
IM-0001-00%s-icontour-manual.txt' % (folder,str(n))

In [ ]:
for pic_path in pic_arr:
    contour = []
    file = open(text_path, 'r') 
    for line in file: 
        contour.append(tuple(map(float, line.split())))
    contour = np.array(contour)

    image_part = dicom.read_file(pic_path).pixel_array  
    plt.figure(figsize=(10,10))
    plt.imshow(image_part, cmap='gray')
    plt.plot(*zip(*contour), )
    plt.title(pic_path.split('/')[5])
    plt.show()

### Plot do ROI de um Contorno

In [ ]:
case = 'SC-HF-NI-36'
series = {'SC-N-40' : '0944',
          'SC-N-03' : '0915',
          'SC-N-02' : '0898',
          'SC-HYP-40' : '0755',
          'SC-HYP-38' : '0734',
          'SC-HYP-03' : '0650',
          'SC-HYP-01' : '0550',
          'SC-HF-NI-36' : '0474',
          'SC-HF-NI-34' : '0446',
          'SC-HF-NI-04' : '0501',
          'SC-HF-NI-03' : '0379',
          'SC-HF-I-40' : '0134',
          'SC-HF-I-04' : '0116',
          'SC-HF-I-02' : '0106',
          'SC-HF-I-01' : '0004'}

image_path_base = './Data/challenge_training/%s/IM-%s' % (case, series[case])
contour_path_base = './Data/Sunnybrook Cardiac MR Database ContoursPart3/\
TrainingDataContours/%s/contours-manual/IRCCI-expert/' % case
contours_list = glob.glob(contour_path_base + '*')
contours_list_series = [k.split('/')[7].split('-')[2] for k in contours_list]
contour_chosen = np.random.choice(contours_list_series)
idx_contour = contours_list_series.index(contour_chosen)
contour_path = contours_list[idx_contour]
image_path = image_path_base + '-%s.dcm' % contour_chosen

In [ ]:
image_part = dicom.read_file(image_path).pixel_array  
contour = []
file = open(contour_path, 'r') 
for line in file: 
    contour.append(tuple(map(float, line.split())))
contour = np.array(contour)

# ROI 
X_min, Y_min = contour[:,0].min(), contour[:,1].min()
X_max, Y_max = contour[:,0].max(), contour[:,1].max()
w = contour[:,0].max() - contour[:,0].min() 
h = contour[:,1].max() - contour[:,1].min()

# Mask
mask_roi = np.zeros(image_part.shape)
mask_roi[int(contour[:,1].min()):int(contour[:,1].max()), 
         int(contour[:,0].min()):int(contour[:,0].max())] = 1.0

# Canvas
x, y = np.meshgrid(np.arange(256), np.arange(256)) 
x, y = x.flatten(), y.flatten()
points = np.vstack((x,y)).T 
p = Path(contour) # make a polygon
grid = p.contains_points(points)
mask_contour = grid.reshape(256,256)
  
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(15, 15))
ax[0, 0].imshow(image_part, cmap='gray')
ax[0, 0].plot(*zip(*contour))
rect = patches.Rectangle((X_min,Y_min), w, h, linewidth=1, edgecolor='r', facecolor='none')
ax[0, 0].add_patch(rect)

ax[1, 0].imshow(mask_roi, cmap='gray')
ax[1, 1].imshow(mask_contour*1, cmap='gray')
ax[0, 1].imshow(image_part, cmap='gray')
ax[0, 1].plot(*zip(*contour))
ax[0, 1].set_xlim(int(contour[:,0].min()), int(contour[:,0].max()))
t = ax[0, 1].set_ylim(int(contour[:,1].min()), int(contour[:,1].max()))

In [ ]:
image_part.shape

In [ ]:
# Plot
f, ax = plt.subplots()
ax.imshow(image_part, cmap='gist_yarg')
#plt.plot(*zip(*contour))
rect = patches.Rectangle(
        (X_min,Y_min),
        w, h,
        linewidth=1, edgecolor='r', facecolor='none')
ax.add_patch(rect)

In [ ]:
cv2.imwrite('Resultado/images/input_image.png', image_part)

# Fim